# LCA of Germany's grid status quo

In [ ]:
import bw2data as bd
import bw2calc as bc

In [ ]:
bd.projects.set_current("paper_plca_grid_expansion_rev")

In addition to the cumulated LCIA results, we want to investigate the contributions of certain grid components, materials and of the underlying directly emitting processes.

## Contributions of components and materials


Setup:

In [ ]:
status_quo_node = bd.get_node(database="grid_status_quo", name="grid_status_quo")
fu = {status_quo_node: 1}

method = ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')

Now we calculate the LCI the standard way, but factorize the matrices because we will do additional calculations with the same matrices in the next step.

In [ ]:
lca = bc.LCA(fu, method=method)
lca.lci(factorize=True)

Calculating the LCIA results for the component and material levels:

In [ ]:
component_scores = {}
for component in status_quo_node.technosphere():
    material_scores = {}
    for material in component.input.technosphere():
        lca.lcia(demand={material.input.id: material.amount * component.amount})
        material_scores[material.input["name"]] = lca.score
    component_scores[component.input["name"]] = material_scores

Saving the results in a DataFrame and cleaning up the labels:

In [ ]:
import pandas as pd

def map_labels(labels, groups, other_label="other"):
    new_labels = {}
    for label in labels:
        for key, group in groups.items():
            if key in label:
                new_labels[label] = group
                break
        else:
            new_labels[label] = other_label
    return new_labels

component_groups = {
    "Overhead line": "overhead lines",
    "cable": "cables",
    "Transformer": "transformers",
    "switchgear": "switchgears",
    "Substation": "substations",
}

material_groups = {
    "aluminium": "aluminum",
    "copper": "copper",
    "iron": "iron & steel",
    "steel": "iron & steel",
    "concrete": "concrete",
    "cement": "concrete",
    "sulfur hexafluoride": "SF6",
    "polyethylene": "plastics",
    "polypropylene": "plastics",
    "plastic": "plastics",
}

df = pd.DataFrame(component_scores)

df.index = df.index.map(
    map_labels(df.index, material_groups, other_label="other materials")
)
df.columns = df.columns.map(map_labels(df.columns, component_groups))

df_components_to_materials = df.groupby(level=0).sum().T.groupby(level=0).sum().T
df_components_to_materials


## Process contributions

To get the information which processes are the direct emitters, but keep the connection to the responsible materials, we use the initially created "aggregated material" activities that essentially sum up the demands for a certain material across all grid components.


First, we collect the aggregated materials nodes:

In [ ]:
aggregated_material_names = [
    "aggregated material: aluminium",
    "aggregated material: steel",
    "aggregated material: concrete",
    "aggregated material: copper",
    "aggregated material: plastics",
    "aggregated material: other",
]

aggregated_material_nodes = [
    bd.get_node(name=name) for name in aggregated_material_names
]

Now, we do the individual LCAs, and aggregate the results by the reference product of the underlying directly emitting processes:

In [ ]:
import bw2analyzer as ba

lca = bc.LCA(
    {aggregated_material_nodes[0]: 1}, method=method
)  # just to build the matrices
lca.lci(factorize=True)
material_top_processes = {}
for mat in aggregated_material_nodes:
    lca.lcia(demand={mat.id: 1})
    top_processes = ba.ContributionAnalysis().annotated_top_processes(lca, limit=5000)
    score_and_product = {}
    for process in top_processes:
        if process[2]["reference product"] in score_and_product:
            score_and_product[process[2]["reference product"]] += process[0]
        else:
            score_and_product[process[2]["reference product"]] = process[0]
    material_top_processes[mat["name"]] = score_and_product

We save the results in a DataFrame again and clean up the names:

In [ ]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminum (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials = pd.DataFrame(material_top_processes)

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

In [ ]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminum (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials = pd.DataFrame(material_top_processes)

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

## Prepare sankey diagram

Now we have all the information we need, we just need to get the data in the right format to feed our Sankey Diagram.


In [ ]:
sankey_data = []  # will be filled with tuples of (source, target, value)

Grid to components:


In [ ]:
for component, score in df_components_to_materials.sum().items():
    sankey_data.append((component, "grid status quo", score))

Components to materials:


In [ ]:
for target, values in df_components_to_materials.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Materials to processes:


In [ ]:
for target, values in df_materials_to_processes.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Create a DataFrame and save as .csv:

In [ ]:
df_sankey = pd.DataFrame(sankey_data, columns=["source", "target", "value"])
df_sankey.value = df_sankey.value / 1e9
df_sankey.columns = ["source", "target", "value"]
df_sankey.to_csv("data/results/sankey_data.csv", index=False)

In [ ]:
df_sankey

The final figure for the paper was created using the Javascript library D3Blocks. The code is available on Observable: https://observablehq.com/@timo-diepers-ws/sankey-grid-status-quo-de. 